In [ ]:
%load_ext rpy2.ipython
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%%R
install.packages("/content/gdrive/My Drive/Packages/memisc_0.99.27.3.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/prodlim_2019.11.13.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/ipred_0.9-11.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/recipes_0.1.16.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/bitops_1.0-7.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/gtools_3.9.2.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/caTools_1.18.2.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/gplots_3.1.1.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/ROCR_1.0-11.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/dplyr_1.0.7.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/proxy_0.4-26.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/e1071_1.7-8.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/randomForest_4.6-14.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/haven_2.4.3.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/ggplot2_3.3.5.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/abind_1.4-5.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/zoo_1.8-9.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/xts_0.12.1.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/TTR_0.24.2.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/quantmod_0.4.18.tar.gz", repos = NULL, type = "source")
install.packages("/content/gdrive/My Drive/Packages/DMwR_0.4.1.tar.gz", repos = NULL, type = "source")

In [ ]:
%%R
#Load packages
library(memisc)
library(ROCR)
library(dplyr)
library(e1071)
library(randomForest)
library(haven)
library(ggplot2)
library(DMwR)

In [ ]:
%%R
install.packages('caret')
library('pROC')
library('caret')

In [ ]:
%%R
#Load data
Test  <- read.csv('/content/gdrive/My Drive/Data/TEST DATA.csv')
Train <- read.csv('/content/gdrive/My Drive/Data/TRAIN PERCOBAAN 1.csv')

In [ ]:
%%R
#Load data
Test  <- read.csv('/content/gdrive/My Drive/Data/DATA Agustus 2021/1.TEST DATA.csv')
Train <- read.csv('/content/gdrive/My Drive/Data/DATA Agustus 2021/2.OR TRAIN DATA.csv')

In [ ]:
%%R
names(Test)[12] <- "Y"
names(Train)[12] <- "Y"
auc_graphF <- function(TPR,FPR,AUC,name,code) {

      H <- data.frame(FPR = FPR, TPR = TPR)
      X <- ggplot() + 
        geom_line(data = H, aes(x = FPR, y = TPR)) + 
        theme(plot.title = element_text(hjust = 0.5), plot.subtitle = element_text(hjust = 0.5), panel.grid.major = element_line(color  = "lightgray", 
              size = 0.25), panel.background = element_rect(fill = "white", color = "black", size = 2)) + 
        labs(x = "False Positive Rate", y = "True Positive Rate", title = paste("Area Under the ROC Curve =",round(AUC*100,digits = 2),"%"), subtitle = paste("Random Forest with",name,"&",code))
      
      ggsave(plot = X, filename = paste("/content/gdrive/My Drive/Result/",name," ",code," AUC.jpg",sep = ""))
    
}

RandomForestX <- function(train, test, ntree, mtry){
  for (i in 1:dim(test)[2]) {
    train[,i] <- as.factor(train[,i])
    test[,i] <- as.factor(test[,i])
  }
  
  #Declaration metrics matrix
  METRIC <- matrix(0,length(ntree)*length(mtry),20)
  colnames(METRIC) <- c("Accuracy","Kappa","AccuracyLower","AccuracyUpper","AccuracyNull","AccuracyPValue","McnemarPValue",
                        "Sensitivity","Specificity","Pos Pred Value","Neg Pred Value","Precision","Recall","F1","Prevalence",
                        "Detection Rate","Detection Prevalence","Balanced Accuracy","AUC","OOB")
  CFM <- list()
  nameX <- paste(ntree,"Trees")
  mtryX <- paste(mtry,"Mtry")
  
  #Random Forest Process
  i <- 0
  k <- 0
  for (ntrees in ntree) { #Looping with different trees
    i <- i + 1
    j <- 0
    for (mtrys in mtry) { #Looping with different m value (selected random variable)
      j <- j + 1
      k <- k + 1
      #Random Forest function
      rf <- randomForest(Y ~., data = train, ntree = ntrees, mtry = mtrys, replace = T, samplesize = nrow(train))
      
      #Y-prediction (label)
      pred <- predict(rf, newdata = test[-dim(train)[2]])
      
      #Y-prediction probability
      pred_prob <- predict(rf, newdata = test[-dim(train)[2]], type = "prob")
      
      #Confusion Matrix
      cfmRF <- confusionMatrix(pred,test$Y)
      
      #Save the metric value
      METRIC[k,] <- c(cfmRF$overall,cfmRF$byClass,auc(roc(test$Y,pred_prob[,2])),mean(rf$err.rate[,1]))

      #ROC Value for ROC & AUC Graph
      X <- prediction(pred_prob[,2],test$Y)
      R <- performance(X,"tpr","fpr")

      auc_graphF(R@y.values[[1]],R@x.values[[1]],METRIC[k,19],nameX[i],mtryX[j])
      
      #Save Confusion Matrix
      CFM[[paste(nameX[i],mtryX[j])]] <- cfmRF$table
      
      #Erase the value for save memory
      rf <- 0
      cfmRF <- 0
      pred <- 0
      pred_prob <- 0
      X <- 0
      R <- 0
    }
  }

  write.csv(METRIC, file = "/content/gdrive/My Drive/Result/Metric.csv", row.names = F)
  write.csv(CFM, file = "/content/gdrive/My Drive/Result/Confusion Matrix.csv", row.names = F)
  return(0)
}

In [ ]:
%%R
result <- RandomForestX(train = Train, test = Test, ntree = c(25,50,75,100,125,150), mtry = c(1,3,5,7,9))